In [7]:
# 从数据库导入数据dfpbroech6
import pandas as pd
from sqlalchemy import create_engine

# 根据你的实际数据库信息填写
username = "panjinhe"
password = "20020112p"
host = "localhost"
port = "5432"
database = "pbroe"

# 定义要查询的表和schema
table_name = 'pbroech6'
schema_name = 'pbroe'

# 定义日期范围
start_date = '2005-04'
end_date = '2025-03'

# 构建连接字符串
connection_string = f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}"

# 创建引擎
engine = create_engine(connection_string)

# 构建带有日期范围筛选的SQL查询语句
# 这样可以在数据库层面直接过滤，效率更高
sql_query = f"""
SELECT * FROM {schema_name}.{table_name}
WHERE "trdmnt" >= '{start_date}' AND "trdmnt" <= '{end_date}'
"""
# 使用 pd.read_sql_query 执行带有条件的查询
dfpbroech6 = pd.read_sql_query(sql_query, engine)

# print("\n数据加载成功！")
# # --- 3. 显示数据信息 ---
# print("\nDataFrame Info:")
print(dfpbroech6.info())
# print("\nDataFrame Head (first 10 rows):")
#display(dfpbroech6.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 666516 entries, 0 to 666515
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   stkcd          666516 non-null  object 
 1   trdmnt         666516 non-null  object 
 2   accper         666516 non-null  object 
 3   shortname      666516 non-null  object 
 4   if_st          666516 non-null  int64  
 5   indcd1         666516 non-null  object 
 6   indnme1        666516 non-null  object 
 7   price          666516 non-null  float64
 8   market_cap     666516 non-null  float64
 9   PB             664633 non-null  float64
 10  total_shares   666516 non-null  float64
 11  eps_ttm_core   611261 non-null  float64
 12  eps_ttm_total  635140 non-null  float64
 13  roe_ttm        633763 non-null  float64
 14  total_equity   666516 non-null  float64
 15  ar             666516 non-null  float64
 16  inventory      666516 non-null  float64
 17  intangibles    666516 non-nul

In [8]:
import pandas as pd
from pathlib import Path

def calculate_roe_average_from_df(df_input: pd.DataFrame, output_path: Path):
    """
    从一个DataFrame计算ROE的滚动平均值，并将结果保存到CSV文件。
    此函数假定输入的DataFrame已经包含了所需的数据。

    Args:
        df_input (pd.DataFrame): 包含 'stkcd', 'trdmnt', 'roe_ttm' 列的DataFrame。
        output_path (Path): 保存结果的CSV文件路径。
    """
    print("--- 开始基于提供的DataFrame计算多年平均ROE ---")

    # 创建一个副本以避免修改原始的 dfpbroech6
    df = df_input.copy()

    print("\n--- 步骤 1: 数据预处理 ---")
    df['stkcd'] = df['stkcd'].astype(str).str.zfill(6)
    # 确保 trdmnt 是 datetime 类型
    df['trdmnt'] = pd.to_datetime(df['trdmnt'])
    df['year'] = df['trdmnt'].dt.year
    df['month'] = df['trdmnt'].dt.month

    # 筛选出 roe_ttm 非空的数据
    df.dropna(subset=['roe_ttm'], inplace=True)
    print("数据预处理完成。")

    print("\n--- 步骤 2: 提取唯一的年度 ROE 数据 ---")
    # 使用 groupby().last() 确保 (stkcd, year) 组合的唯一性
    print(f"提取到 {len(df[df['month'] == 5])} 条5月份的ROE记录。")
    annual_roe = df[df['month'] == 5].groupby(['stkcd', 'year'], as_index=False).last()
    print(f"通过 Groupby 操作，保留 {len(annual_roe)} 条唯一的年度ROE记录用于计算。")

    print("\n--- 步骤 3: 计算滚动平均ROE ---")
    # 为了使用 rolling，需要将 year 设置为索引的一部分，并确保数据按时间排序
    annual_roe.sort_values(by=['stkcd', 'year'], inplace=True)
    annual_roe.set_index('year', inplace=True)

    # 定义要计算的窗口期
    windows = {'roe_avg_2y': 2, 'roe_avg_3y': 3, 'roe_avg_5y': 5}

    # 使用 groupby().rolling() 计算平均值
    avg_list = []
    for col_name, window_size in windows.items():
        # 对每个 stkcd 分组，然后在 year 索引上进行滚动计算
        avg = annual_roe.groupby('stkcd')['roe_ttm'].rolling(
            window=window_size,
            min_periods=window_size # 确保有足够的数据点
        ).mean()
        avg.name = col_name # 重命名 Series
        avg_list.append(avg)
        print(f"已计算 {window_size} 年滚动平均ROE。")

    # 合并所有平均值结果
    merged_avg = pd.concat(avg_list, axis=1).reset_index()
    print("已合并所有平均ROE数据。")

    print("\n--- 步骤 4: 将平均ROE数据合并回原始月度数据 ---")
    # 将年度平均ROE数据合并到原始的月度DataFrame中
    final_df = pd.merge(df, merged_avg, on=['stkcd', 'year'], how='left')
    print("合并完成。")

    print(f"\n--- 步骤 5: 保存结果到文件 ---")
    try:
        # 确保输出目录存在
        output_path.parent.mkdir(parents=True, exist_ok=True)
        final_df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"结果已成功保存到: {output_path}")
        print("\n最终数据预览 (包含平均ROE的非空行):")
        # 显示包含计算结果的尾部数据作为示例
        preview_df = final_df.dropna(subset=['roe_avg_2y', 'roe_avg_3y', 'roe_avg_5y'])
        print(preview_df[['stkcd', 'trdmnt', 'roe_ttm', 'roe_avg_2y', 'roe_avg_3y', 'roe_avg_5y']].tail())
    except Exception as e:
        print(f"保存文件失败: {e}")


# --- 如何在您的环境中使用 ---

# 假设您的 dfpbroech6 已经加载完毕
# 例如:
# dfpbroech6 = pd.read_sql_query(...) or pd.read_csv(...)

# 1. 定义输出文件路径
output_file = Path('pbroe6.2_ROEavg.csv')

# 2. 调用函数，传入您的DataFrame
# 确保 dfpbroech6 变量存在且已加载数据
if 'dfpbroech6' in locals() and isinstance(dfpbroech6, pd.DataFrame):
    calculate_roe_average_from_df(dfpbroech6, output_file)
else:
    print("错误：请先确保您的数据已加载到名为 'dfpbroech6' 的DataFrame中。")



--- 开始基于提供的DataFrame计算多年平均ROE ---

--- 步骤 1: 数据预处理 ---
数据预处理完成。

--- 步骤 2: 提取唯一的年度 ROE 数据 ---
提取到 51731 条5月份的ROE记录。
通过 Groupby 操作，保留 51731 条唯一的年度ROE记录用于计算。

--- 步骤 3: 计算滚动平均ROE ---
已计算 2 年滚动平均ROE。
已计算 3 年滚动平均ROE。
已计算 5 年滚动平均ROE。
已合并所有平均ROE数据。

--- 步骤 4: 将平均ROE数据合并回原始月度数据 ---
合并完成。

--- 步骤 5: 保存结果到文件 ---
结果已成功保存到: pbroe6.2_ROEavg.csv

最终数据预览 (包含平均ROE的非空行):
         stkcd     trdmnt   roe_ttm  roe_avg_2y  roe_avg_3y  roe_avg_5y
633758  000049 2012-11-01  0.488352    0.447708    0.266827    0.158547
633759  000049 2016-02-01  0.366208    0.310620    0.359984    0.385792
633760  000049 2019-11-01  0.262240    0.226990    0.241607    0.269213
633761  000049 2020-12-01  0.253250    0.243929    0.232562    0.242134
633762  000049 2024-02-01  0.141541    0.140773    0.176110    0.209560


In [13]:
# 计算残差
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pathlib import Path
import os
from joblib import Parallel, delayed

# =================================================================== #
#                           【1. 配置区域】                           #
# =================================================================== #
# 输入文件：包含多年平均ROE的数据文件
INPUT_FILE = Path('./pbroe6.2_ROEavg.csv')

# 输出文件：保存最终结果
OUTPUT_DIR = Path('./')
OUTPUT_FILENAME = 'pbroe6.2_avg_roe_residuals.csv'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 定义要处理的多年平均ROE列
AVG_ROE_COLS = ['roe_avg_2y', 'roe_avg_3y', 'roe_avg_5y']

# 时序分位数计算周期
QUANTILE_PERIODS = [10]


# =================================================================== #
#                       【2. 核心计算函数】                         #
# =================================================================== #

def run_regression_for_avg_roe(df: pd.DataFrame, avg_roe_col: str) -> pd.DataFrame:
    """
    对给定的多年平均ROE列执行月度行业内回归，并计算标准化残差。
    【关键修复点】此函数现在只返回包含键和新残差列的精简DataFrame。
    """
    print(f"\n--- 开始为 '{avg_roe_col}' 执行回归 ---")

    df_regr = df.copy()
    df_regr.dropna(subset=[avg_roe_col, 'PB', 'indnme1'], inplace=True)
    df_regr = df_regr[df_regr['PB'] > 0]
    df_regr['lnPB'] = np.log(df_regr['PB'])

    if df_regr.empty:
        print(f"警告: 为 '{avg_roe_col}' 准备数据后，无有效记录可供回归。")
        return pd.DataFrame()

    def calculate_residuals(group, roe_col):
        if len(group) < 10:
            group['residual_zscore'] = np.nan
            return group
        try:
            y = group['lnPB']
            X = sm.add_constant(group[roe_col])
            model = sm.OLS(y, X, missing='drop').fit()
            residuals = model.resid
            resid_mean, resid_std = residuals.mean(), residuals.std()
            if resid_std > 1e-6:
                group['residual_zscore'] = (residuals - resid_mean) / resid_std
            else:
                group['residual_zscore'] = 0.0
        except Exception:
            group['residual_zscore'] = np.nan
        return group

    results = df_regr.groupby(['trdmnt', 'indnme1']).apply(calculate_residuals, roe_col=avg_roe_col, include_groups=False)
    results.dropna(subset=['residual_zscore'], inplace=True)

    new_resid_col_name = f"residual_zscore_{avg_roe_col.replace('roe_', '')}"
    results.rename(columns={'residual_zscore': new_resid_col_name}, inplace=True)

    print(f"回归完成，为 '{avg_roe_col}' 计算出 {len(results)} 条有效残差。")

    # 【关键修复点】只返回包含关键信息和新残差的精简DataFrame
    final_results = results.reset_index()
    return final_results[['stkcd', 'trdmnt', new_resid_col_name]]


def calculate_quantiles_for_single_stock(stock_df, residual_col, periods):
    stock_df = stock_df.sort_values(by='trdmnt').copy()
    for period in periods:
        quantile_col_name = f"residual_quantile_{period}m_{residual_col.split('_')[-1]}"
        stock_df[quantile_col_name] = stock_df[residual_col].rolling(
            window=period, min_periods=1
        ).apply(lambda y: y.rank(pct=True).iloc[-1], raw=False)
    return stock_df


def calculate_time_series_quantiles_parallel(df: pd.DataFrame, residual_col: str, periods: list) -> pd.DataFrame:
    print(f"\n--- 开始为 '{residual_col}' 计算时序分位数 (并行) ---")

    df_quant = df[['stkcd', 'trdmnt', residual_col]].dropna().copy()
    if df_quant.empty:
        print(f"警告: 列 '{residual_col}' 没有有效数据，无法计算分位数。")
        return pd.DataFrame()

    num_cores = os.cpu_count() or 1
    print(f"使用 {num_cores} 个CPU核心进行并行计算。")

    grouped_dfs = [group for _, group in df_quant.groupby('stkcd')]

    results = Parallel(n_jobs=-1)(
        delayed(calculate_quantiles_for_single_stock)(stock_df, residual_col, periods)
        for stock_df in grouped_dfs
    )

    df_quantiles = pd.concat(results).sort_values(by=['stkcd', 'trdmnt']).reset_index(drop=True)
    print(f"时序分位数计算完成。")
    return df_quantiles


# =================================================================== #
#                          【3. 主函数执行】                          #
# =================================================================== #

def main():
    print(f"--- 开始处理文件: {INPUT_FILE} ---")
    try:
        df_main = pd.read_csv(INPUT_FILE)
        df_main['trdmnt'] = pd.to_datetime(df_main['trdmnt']).dt.to_period('M').astype(str)
        df_main['stkcd'] = df_main['stkcd'].astype(str).str.zfill(6)
    except FileNotFoundError:
        print(f"错误: 找不到输入文件 {INPUT_FILE}。请检查文件路径。")
        return

    for avg_roe_col in AVG_ROE_COLS:
        # 1. 计算回归残差
        df_residuals = run_regression_for_avg_roe(df_main, avg_roe_col)

        if df_residuals.empty:
            continue

        # 【关键修复点】合并逻辑现在更简单、更安全
        df_main = pd.merge(df_main, df_residuals, on=['stkcd', 'trdmnt'], how='left')

        # 2. 基于新残差计算时序分位数
        # `df_residuals`现在只包含一个残差列，所以可以直接获取其列名
        new_resid_col_name = [col for col in df_residuals.columns if 'residual_zscore' in col][0]
        df_quantiles = calculate_time_series_quantiles_parallel(df_main, new_resid_col_name, QUANTILE_PERIODS)

        if df_quantiles.empty:
            continue

        df_quantiles.drop(columns=[new_resid_col_name], inplace=True)
        df_main = pd.merge(df_main, df_quantiles, on=['stkcd', 'trdmnt'], how='left')

    output_path = OUTPUT_DIR / OUTPUT_FILENAME
    try:
        final_columns = ['stkcd', 'trdmnt'] + [col for col in df_main.columns if col not in ['stkcd', 'trdmnt']]
        df_main = df_main[final_columns]

        df_main.to_csv(output_path, index=False, encoding='utf-8-sig', float_format='%.6f')
        print(f"\n--- 全部计算完成！---")
        print(f"最终数据已保存至: {output_path}")
        print("\n最终文件内容预览:")
        print(df_main.dropna(subset=[col for col in df_main.columns if 'quantile' in col]).tail())
    except Exception as e:
        print(f"保存文件时出错: {e}")

if __name__ == "__main__":
    main()


--- 开始处理文件: pbroe6.2_ROEavg.csv ---

--- 开始为 'roe_avg_2y' 执行回归 ---
回归完成，为 'roe_avg_2y' 计算出 526734 条有效残差。

--- 开始为 'residual_zscore_avg_2y' 计算时序分位数 (并行) ---
使用 192 个CPU核心进行并行计算。
时序分位数计算完成。

--- 开始为 'roe_avg_3y' 执行回归 ---
回归完成，为 'roe_avg_3y' 计算出 468993 条有效残差。

--- 开始为 'residual_zscore_avg_3y' 计算时序分位数 (并行) ---
使用 192 个CPU核心进行并行计算。
时序分位数计算完成。

--- 开始为 'roe_avg_5y' 执行回归 ---
回归完成，为 'roe_avg_5y' 计算出 368887 条有效残差。

--- 开始为 'residual_zscore_avg_5y' 计算时序分位数 (并行) ---
使用 192 个CPU核心进行并行计算。
时序分位数计算完成。

--- 全部计算完成！---
最终数据已保存至: pbroe6.2_avg_roe_residuals.csv

最终文件内容预览:
         stkcd   trdmnt      accper shortname  if_st indcd1     indnme1  \
633758  000049  2012-11  2012-09-30      德赛电池      0    C38  电气机械和器材制造业   
633759  000049  2016-02  2015-09-30      德赛电池      0    C38  电气机械和器材制造业   
633760  000049  2019-11  2019-09-30      德赛电池      0    C38  电气机械和器材制造业   
633761  000049  2020-12  2020-09-30      德赛电池      0    C38  电气机械和器材制造业   
633762  000049  2024-02  2023-09-30      德赛电池      0    C38  电气机械

In [18]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from joblib import Parallel, delayed

def build_pbroe6_2_portfolios(
    data_file: Path,
    strategy_configs: dict,
    cs_quantile: float = 0.2,
    ts_threshold: float = 0.2
) -> dict:
    """
    基于多年平均ROE的残差和时序分位数构建投资组合。

    Args:
        data_file (Path): 包含所有预计算指标的数据文件。
        strategy_configs (dict): 策略配置，映射策略名到残差和分位数。
        cs_quantile (float): 横截面残差的筛选分位数阈值。
        ts_threshold (float): 时序残差分位数的筛选阈值。

    Returns:
        dict: 一个嵌套字典，结构为 {'策略名': {调仓日期: {股票代码集合}}}。
    """
    print(f"--- 步骤 1: 构建 PBROE 6.2 系列投资组合 ---")
    try:
        df = pd.read_csv(data_file)
        df['trdmnt'] = pd.to_datetime(df['trdmnt'])
        # 创建用于选股的调入日期
        df['调入日期'] = df['trdmnt'] + pd.DateOffset(months=1)
        df['stkcd'] = df['stkcd'].astype(str).str.zfill(6)
        print(f"数据文件 '{data_file.name}' 加载成功。")
    except FileNotFoundError:
        print(f"错误: 找不到数据文件 {data_file}。")
        return {}

    all_selections = {}
    for strategy_name, configs in strategy_configs.items():
        resid_col = configs['residual']
        quant_col = configs['quantile']
        print(f"  -> 正在为策略 '{strategy_name}' 构建持仓...")

        df_strategy = df[['调入日期', 'stkcd', resid_col, quant_col]].dropna().copy()

        selections_dict = {}
        for date, group in df_strategy.groupby('调入日期'):
            resid_threshold = group[resid_col].quantile(cs_quantile)
            cs_selected = set(group[group[resid_col] <= resid_threshold]['stkcd'])

            ts_selected = set(group[group[quant_col] <= ts_threshold]['stkcd'])

            final_selection = cs_selected.intersection(ts_selected)

            if final_selection:
                selections_dict[date] = final_selection

        all_selections[strategy_name] = selections_dict
        print(f"     已为 {len(selections_dict)} 个调仓日构建投资组合。")

    # 【新增】打印所有策略涉及的唯一调仓日期总数
    total_unique_dates = set()
    for selections in all_selections.values():
        total_unique_dates.update(selections.keys())
    print(f"\n构建投资组合完成。所有策略共涉及 {len(total_unique_dates)} 个唯一的调仓日期。")

    return all_selections


def _backtest_single_strategy_job(strategy_name: str, selections: dict, backtest_months: pd.DatetimeIndex, returns_df: pd.DataFrame) -> pd.Series:
    """
    【新增】用于并行计算的辅助函数，回测单个策略。
    """
    print(f"  -> 开始回测策略: {strategy_name}")
    portfolio_map = pd.Series(index=backtest_months, dtype='object')
    rebalance_dates = sorted(selections.keys())
    for month in backtest_months:
        applicable_date = next((d for d in reversed(rebalance_dates) if d <= month), None)
        if applicable_date:
            portfolio_map[month] = selections[applicable_date]

    monthly_returns = []
    for month, stocks in portfolio_map.items():
        if not stocks:
            monthly_returns.append(0.0)
            continue

        month_str = month.strftime('%Y-%m')
        current_returns = returns_df[
            (returns_df['Trdmnt'] == month_str) &
            (returns_df['Stkcd'].isin(stocks))
        ]
        avg_return = current_returns['Mretwd'].mean() if not current_returns.empty else 0.0
        monthly_returns.append(avg_return)

    print(f"  <- 完成回测策略: {strategy_name}")
    return pd.Series(monthly_returns, index=backtest_months, name=f"return_{strategy_name}")


def run_multi_strategy_backtest_parallel(all_selections: dict, returns_df: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    """
    【重构】对多个策略并行执行向量化回测。
    """
    print("\n--- 步骤 2: 执行多策略向量化回测 (并行加速) ---")
    backtest_months = pd.to_datetime(pd.date_range(start_date, end_date, freq='MS'))

    num_cores = os.cpu_count() or 1
    print(f"使用 {num_cores} 个CPU核心进行并行回测。")

    # 使用joblib并行执行每个策略的回测
    results = Parallel(n_jobs=-1)(
        delayed(_backtest_single_strategy_job)(
            strategy_name,
            selections,
            backtest_months,
            returns_df
        ) for strategy_name, selections in all_selections.items()
    )

    all_returns_df = pd.concat(results, axis=1)
    print(f"回测完成，已为 {len(all_selections)} 个策略生成 {len(all_returns_df)} 条月度收益记录。")
    return all_returns_df


def calculate_and_save_performance_multi(
    all_returns_df: pd.DataFrame,
    benchmark_df: pd.DataFrame,
    all_selections: dict,
    risk_free_rate: float,
    returns_output_file: Path,
    performance_output_file: Path
):
    """
    计算并保存多个策略的详细绩效指标，并格式化输出。
    """
    print("\n--- 步骤 3: 计算并保存各策略绩效指标 ---")

    results = all_returns_df.join(benchmark_df.set_index('date'), how='left').fillna(0)
    all_metrics = []

    for strategy_name, selections in all_selections.items():
        return_col = f'return_{strategy_name}'
        cum_col = f'cum_{strategy_name}'
        results[cum_col] = (1 + results[return_col]).cumprod()

        total_months = len(results)
        final_return = results[cum_col].iloc[-1]
        annualized_return = final_return ** (12 / total_months) - 1
        annualized_volatility = results[return_col].std() * np.sqrt(12)
        sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility if annualized_volatility != 0 else 0

        rolling_max = results[cum_col].expanding().max()
        drawdown = (results[cum_col] - rolling_max) / rolling_max
        max_drawdown = drawdown.min()

        annualized_benchmark_return = (1 + results['benchmark_return']).prod() ** (12 / total_months) - 1
        excess_return = results[return_col] - results['benchmark_return']
        annualized_excess_return = annualized_return - annualized_benchmark_return
        tracking_error = excess_return.std() * np.sqrt(12)
        information_ratio = annualized_excess_return / tracking_error if tracking_error != 0 else 0

        turnover_list = []
        rebalance_dates = sorted(selections.keys())
        for i in range(1, len(rebalance_dates)):
            prev_portfolio = selections.get(rebalance_dates[i-1], set())
            curr_portfolio = selections.get(rebalance_dates[i], set())
            if not prev_portfolio: continue
            stocks_sold = len(prev_portfolio - curr_portfolio)
            period_turnover = stocks_sold / len(prev_portfolio)
            turnover_list.append(period_turnover)
        annual_turnover = np.mean(turnover_list) * 12 if turnover_list else 0.0

        metrics = {
            '策略名称': strategy_name,
            '年化收益率': annualized_return, '年化波动率': annualized_volatility, '夏普比率': sharpe_ratio,
            '最大回撤': max_drawdown, '年化换手率': annual_turnover, '累计收益率': final_return - 1,
            '年化超额收益率': annualized_excess_return, '信息比率': information_ratio, '跟踪误差': tracking_error,
        }
        all_metrics.append(metrics)

    performance_df = pd.DataFrame(all_metrics).set_index('策略名称')
    performance_df.loc['基准 (沪深300)', '年化收益率'] = annualized_benchmark_return

    results.to_csv(returns_output_file, encoding='utf-8-sig', float_format='%.6f')
    print(f"月度收益数据已保存至: {returns_output_file}")

    formatted_performance_df = performance_df.copy()
    percent_cols = ['年化收益率', '年化波动率', '最大回撤', '年化换手率', '累计收益率', '年化超额收益率', '跟踪误差']
    for col in percent_cols:
        formatted_performance_df[col] = formatted_performance_df[col].apply(lambda x: f"{x:.2%}" if pd.notna(x) else '-')

    float_cols = ['夏普比率', '信息比率']
    for col in float_cols:
        formatted_performance_df[col] = formatted_performance_df[col].apply(lambda x: f"{x:.2f}" if pd.notna(x) else '-')

    formatted_performance_df.to_csv(performance_output_file, encoding='utf-8-sig')
    print(f"格式化的绩效评估报告已保存至: {performance_output_file}")

    print("\n--- 各策略绩效对比简报 ---")
    print(formatted_performance_df)


def main():
    """主执行函数"""
    # --- 配置区域 ---
    DATA_PATH = Path("E:/PBROE/data")
    CH6_PATH = Path("E:/PBROE/ch6")

    DATA_FILE = CH6_PATH / 'pbroe6.2_avg_roe_residuals.csv'
    RETURNS_FILE = DATA_PATH / 'TRDNEW_Mnth.csv'
    BENCHMARK_FILE = DATA_PATH / 'benchmark_indices.csv'

    RETURNS_OUTPUT_FILE = CH6_PATH / 'pbroe6.2_returns.csv'
    PERFORMANCE_OUTPUT_FILE = CH6_PATH / 'pbroe6.2_performance.csv'

    STRATEGY_CONFIGS = {
        'pbroe6.2_avg_2y': {
            'residual': 'residual_zscore_avg_2y',
            'quantile': 'residual_quantile_10m_2y'
        },
        'pbroe6.2_avg_3y': {
            'residual': 'residual_zscore_avg_3y',
            'quantile': 'residual_quantile_10m_3y'
        },
        'pbroe6.2_avg_5y': {
            'residual': 'residual_zscore_avg_5y',
            'quantile': 'residual_quantile_10m_5y'
        }
    }

    BACKTEST_START_DATE = '2010-05-01'
    BACKTEST_END_DATE = '2025-04-30'
    BENCHMARK_CODE = '000300'
    RISK_FREE_RATE = 0.03

    try:
        all_selections = build_pbroe6_2_portfolios(DATA_FILE, STRATEGY_CONFIGS)
        if not all_selections: return

        returns_df = pd.read_csv(RETURNS_FILE)
        returns_df['Stkcd'] = returns_df['Stkcd'].astype(str).str.zfill(6)
        returns_df['Trdmnt'] = pd.to_datetime(returns_df['Trdmnt']).dt.strftime('%Y-%m')
        returns_df['Mretwd'] = pd.to_numeric(returns_df['Mretwd'], errors='coerce').fillna(0)

        all_benchmarks_df = pd.read_csv(BENCHMARK_FILE)
        benchmark_df = all_benchmarks_df[all_benchmarks_df['Indexcd'].astype(str).str.zfill(6) == BENCHMARK_CODE].copy()
        benchmark_df['date'] = pd.to_datetime(benchmark_df['Month'], format='%Y-%m')
        benchmark_df.rename(columns={'Idxrtn': 'benchmark_return'}, inplace=True)
        benchmark_df = benchmark_df[['date', 'benchmark_return']]

        # 【更新】调用并行版本的回测函数
        all_returns_df = run_multi_strategy_backtest_parallel(all_selections, returns_df, BACKTEST_START_DATE, BACKTEST_END_DATE)

        calculate_and_save_performance_multi(
            all_returns_df, benchmark_df, all_selections, RISK_FREE_RATE,
            RETURNS_OUTPUT_FILE, PERFORMANCE_OUTPUT_FILE
        )
        print("\n--- PBROE 6.2 (基于多年平均ROE) 策略回测完成！ ---")

    except Exception as e:
        import traceback
        print(f"\n执行过程中出现严重错误: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    main()

--- 步骤 1: 构建 PBROE 6.2 系列投资组合 ---
数据文件 'pbroe6.2_avg_roe_residuals.csv' 加载成功。
  -> 正在为策略 'pbroe6.2_avg_2y' 构建持仓...
     已为 224 个调仓日构建投资组合。
  -> 正在为策略 'pbroe6.2_avg_3y' 构建持仓...
     已为 212 个调仓日构建投资组合。
  -> 正在为策略 'pbroe6.2_avg_5y' 构建持仓...
     已为 188 个调仓日构建投资组合。

构建投资组合完成。所有策略共涉及 224 个唯一的调仓日期。

--- 步骤 2: 执行多策略向量化回测 (并行加速) ---
使用 192 个CPU核心进行并行回测。
回测完成，已为 3 个策略生成 180 条月度收益记录。

--- 步骤 3: 计算并保存各策略绩效指标 ---
月度收益数据已保存至: E:\PBROE\ch6\pbroe6.2_returns.csv
格式化的绩效评估报告已保存至: E:\PBROE\ch6\pbroe6.2_performance.csv

--- 各策略绩效对比简报 ---
                  年化收益率   年化波动率  夏普比率     最大回撤    年化换手率    累计收益率 年化超额收益率  \
策略名称                                                                       
pbroe6.2_avg_2y  16.78%  27.00%  0.51  -37.65%  498.50%  924.57%  15.39%   
pbroe6.2_avg_3y  15.14%  26.90%  0.45  -38.57%  492.80%  728.71%  13.75%   
pbroe6.2_avg_5y  14.79%  27.02%  0.44  -38.10%  491.14%  692.03%  13.41%   
基准 (沪深300)        1.39%       -     -        -        -        -       -   

                 信息比

In [20]:
import pandas as pd
import numpy as np
from pathlib import Path
import os
from joblib import Parallel, delayed

def build_pbroe6_2_pure_residual_portfolios(
    data_file: Path,
    strategy_configs: dict,
    cs_quantile: float = 0.1
) -> dict:
    """
    【已修改】仅基于多年平均ROE的残差（横截面信号）构建投资组合。

    Args:
        data_file (Path): 包含所有预计算指标的数据文件。
        strategy_configs (dict): 策略配置，映射策略名到残差列。
        cs_quantile (float): 横截面残差的筛选分位数阈值。

    Returns:
        dict: 一个嵌套字典，结构为 {'策略名': {调仓日期: {股票代码集合}}}。
    """
    print(f"--- 步骤 1: 构建 PBROE 6.2 (纯残差) 系列投资组合 ---")
    try:
        df = pd.read_csv(data_file)
        df['trdmnt'] = pd.to_datetime(df['trdmnt'])
        # 创建用于选股的调入日期
        df['调入日期'] = df['trdmnt'] + pd.DateOffset(months=1)
        df['stkcd'] = df['stkcd'].astype(str).str.zfill(6)
        print(f"数据文件 '{data_file.name}' 加载成功。")
    except FileNotFoundError:
        print(f"错误: 找不到数据文件 {data_file}。")
        return {}

    all_selections = {}
    for strategy_name, configs in strategy_configs.items():
        resid_col = configs['residual']
        print(f"  -> 正在为策略 '{strategy_name}' 构建持仓...")

        # 【修改】现在只依赖残差列，不再需要时序分位数
        df_strategy = df[['调入日期', 'stkcd', resid_col]].dropna().copy()

        selections_dict = {}
        for date, group in df_strategy.groupby('调入日期'):
            # 【修改】只进行横截面筛选
            resid_threshold = group[resid_col].quantile(cs_quantile)
            final_selection = set(group[group[resid_col] <= resid_threshold]['stkcd'])

            if final_selection:
                selections_dict[date] = final_selection

        all_selections[strategy_name] = selections_dict
        print(f"     已为 {len(selections_dict)} 个调仓日构建投资组合。")

    # 打印所有策略涉及的唯一调仓日期总数
    total_unique_dates = set()
    for selections in all_selections.values():
        total_unique_dates.update(selections.keys())
    print(f"\n构建投资组合完成。所有策略共涉及 {len(total_unique_dates)} 个唯一的调仓日期。")

    return all_selections


def _backtest_single_strategy_job(strategy_name: str, selections: dict, backtest_months: pd.DatetimeIndex, returns_df: pd.DataFrame) -> pd.Series:
    """
    用于并行计算的辅助函数，回测单个策略。
    """
    print(f"  -> 开始回测策略: {strategy_name}")
    portfolio_map = pd.Series(index=backtest_months, dtype='object')
    rebalance_dates = sorted(selections.keys())
    for month in backtest_months:
        applicable_date = next((d for d in reversed(rebalance_dates) if d <= month), None)
        if applicable_date:
            portfolio_map[month] = selections[applicable_date]

    monthly_returns = []
    for month, stocks in portfolio_map.items():
        if not stocks:
            monthly_returns.append(0.0)
            continue

        month_str = month.strftime('%Y-%m')
        current_returns = returns_df[
            (returns_df['Trdmnt'] == month_str) &
            (returns_df['Stkcd'].isin(stocks))
        ]
        avg_return = current_returns['Mretwd'].mean() if not current_returns.empty else 0.0
        monthly_returns.append(avg_return)

    print(f"  <- 完成回测策略: {strategy_name}")
    return pd.Series(monthly_returns, index=backtest_months, name=f"return_{strategy_name}")


def run_multi_strategy_backtest_parallel(all_selections: dict, returns_df: pd.DataFrame, start_date: str, end_date: str) -> pd.DataFrame:
    """
    对多个策略并行执行向量化回测。
    """
    print("\n--- 步骤 2: 执行多策略向量化回测 (并行加速) ---")
    backtest_months = pd.to_datetime(pd.date_range(start_date, end_date, freq='MS'))

    num_cores = os.cpu_count() or 1
    print(f"使用 {num_cores} 个CPU核心进行并行回测。")

    results = Parallel(n_jobs=-1)(
        delayed(_backtest_single_strategy_job)(
            strategy_name,
            selections,
            backtest_months,
            returns_df
        ) for strategy_name, selections in all_selections.items()
    )

    all_returns_df = pd.concat(results, axis=1)
    print(f"回测完成，已为 {len(all_selections)} 个策略生成 {len(all_returns_df)} 条月度收益记录。")
    return all_returns_df


def calculate_and_save_performance_multi(
    all_returns_df: pd.DataFrame,
    benchmark_df: pd.DataFrame,
    all_selections: dict,
    risk_free_rate: float,
    returns_output_file: Path,
    performance_output_file: Path
):
    """
    计算并保存多个策略的详细绩效指标，并格式化输出。
    """
    print("\n--- 步骤 3: 计算并保存各策略绩效指标 ---")

    results = all_returns_df.join(benchmark_df.set_index('date'), how='left').fillna(0)
    all_metrics = []

    for strategy_name, selections in all_selections.items():
        return_col = f'return_{strategy_name}'
        cum_col = f'cum_{strategy_name}'
        results[cum_col] = (1 + results[return_col]).cumprod()

        total_months = len(results)
        final_return = results[cum_col].iloc[-1]
        annualized_return = final_return ** (12 / total_months) - 1
        annualized_volatility = results[return_col].std() * np.sqrt(12)
        sharpe_ratio = (annualized_return - risk_free_rate) / annualized_volatility if annualized_volatility != 0 else 0

        rolling_max = results[cum_col].expanding().max()
        drawdown = (results[cum_col] - rolling_max) / rolling_max
        max_drawdown = drawdown.min()

        annualized_benchmark_return = (1 + results['benchmark_return']).prod() ** (12 / total_months) - 1
        excess_return = results[return_col] - results['benchmark_return']
        annualized_excess_return = annualized_return - annualized_benchmark_return
        tracking_error = excess_return.std() * np.sqrt(12)
        information_ratio = annualized_excess_return / tracking_error if tracking_error != 0 else 0

        turnover_list = []
        rebalance_dates = sorted(selections.keys())
        for i in range(1, len(rebalance_dates)):
            prev_portfolio = selections.get(rebalance_dates[i-1], set())
            curr_portfolio = selections.get(rebalance_dates[i], set())
            if not prev_portfolio: continue
            stocks_sold = len(prev_portfolio - curr_portfolio)
            period_turnover = stocks_sold / len(prev_portfolio)
            turnover_list.append(period_turnover)
        annual_turnover = np.mean(turnover_list) * 12 if turnover_list else 0.0

        metrics = {
            '策略名称': strategy_name,
            '年化收益率': annualized_return, '年化波动率': annualized_volatility, '夏普比率': sharpe_ratio,
            '最大回撤': max_drawdown, '年化换手率': annual_turnover, '累计收益率': final_return - 1,
            '年化超额收益率': annualized_excess_return, '信息比率': information_ratio, '跟踪误差': tracking_error,
        }
        all_metrics.append(metrics)

    performance_df = pd.DataFrame(all_metrics).set_index('策略名称')
    performance_df.loc['基准 (沪深300)', '年化收益率'] = annualized_benchmark_return

    results.to_csv(returns_output_file, encoding='utf-8-sig', float_format='%.6f')
    print(f"月度收益数据已保存至: {returns_output_file}")

    formatted_performance_df = performance_df.copy()
    percent_cols = ['年化收益率', '年化波动率', '最大回撤', '年化换手率', '累计收益率', '年化超额收益率', '跟踪误差']
    for col in percent_cols:
        formatted_performance_df[col] = formatted_performance_df[col].apply(lambda x: f"{x:.2%}" if pd.notna(x) else '-')

    float_cols = ['夏普比率', '信息比率']
    for col in float_cols:
        formatted_performance_df[col] = formatted_performance_df[col].apply(lambda x: f"{x:.2f}" if pd.notna(x) else '-')

    formatted_performance_df.to_csv(performance_output_file, encoding='utf-8-sig')
    print(f"格式化的绩效评估报告已保存至: {performance_output_file}")

    print("\n--- 各策略绩效对比简报 ---")
    print(formatted_performance_df)


def main():
    """主执行函数"""
    # --- 配置区域 ---
    DATA_PATH = Path("E:/PBROE/data")
    CH6_PATH = Path("E:/PBROE/ch6")

    DATA_FILE = CH6_PATH / 'pbroe6.2_avg_roe_residuals.csv'
    RETURNS_FILE = DATA_PATH / 'TRDNEW_Mnth.csv'
    BENCHMARK_FILE = DATA_PATH / 'benchmark_indices.csv'

    # 【修改】更新输出文件名以反映新策略
    RETURNS_OUTPUT_FILE = CH6_PATH / 'pbroe6.2_pure_residual_returns.csv'
    PERFORMANCE_OUTPUT_FILE = CH6_PATH / 'pbroe6.2_pure_residual_performance.csv'

    # 【修改】策略配置现在只关心残差列
    STRATEGY_CONFIGS = {
        'pbroe6.2_pure_resid_2y': {
            'residual': 'residual_zscore_avg_2y'
        },
        'pbroe6.2_pure_resid_3y': {
            'residual': 'residual_zscore_avg_3y'
        },
        'pbroe6.2_pure_resid_5y': {
            'residual': 'residual_zscore_avg_5y'
        }
    }

    BACKTEST_START_DATE = '2010-05-01'
    BACKTEST_END_DATE = '2025-04-30'
    BENCHMARK_CODE = '000300'
    RISK_FREE_RATE = 0.03

    try:
        # 【修改】调用新的投资组合构建函数
        all_selections = build_pbroe6_2_pure_residual_portfolios(DATA_FILE, STRATEGY_CONFIGS)
        if not all_selections: return

        returns_df = pd.read_csv(RETURNS_FILE)
        returns_df['Stkcd'] = returns_df['Stkcd'].astype(str).str.zfill(6)
        returns_df['Trdmnt'] = pd.to_datetime(returns_df['Trdmnt']).dt.strftime('%Y-%m')
        returns_df['Mretwd'] = pd.to_numeric(returns_df['Mretwd'], errors='coerce').fillna(0)

        all_benchmarks_df = pd.read_csv(BENCHMARK_FILE)
        benchmark_df = all_benchmarks_df[all_benchmarks_df['Indexcd'].astype(str).str.zfill(6) == BENCHMARK_CODE].copy()
        benchmark_df['date'] = pd.to_datetime(benchmark_df['Month'], format='%Y-%m')
        benchmark_df.rename(columns={'Idxrtn': 'benchmark_return'}, inplace=True)
        benchmark_df = benchmark_df[['date', 'benchmark_return']]

        all_returns_df = run_multi_strategy_backtest_parallel(all_selections, returns_df, BACKTEST_START_DATE, BACKTEST_END_DATE)

        calculate_and_save_performance_multi(
            all_returns_df, benchmark_df, all_selections, RISK_FREE_RATE,
            RETURNS_OUTPUT_FILE, PERFORMANCE_OUTPUT_FILE
        )
        print("\n--- PBROE 6.2 (纯残差) 策略回测完成！ ---")

    except Exception as e:
        import traceback
        print(f"\n执行过程中出现严重错误: {e}")
        traceback.print_exc()

if __name__ == "__main__":
    main()

--- 步骤 1: 构建 PBROE 6.2 (纯残差) 系列投资组合 ---
数据文件 'pbroe6.2_avg_roe_residuals.csv' 加载成功。
  -> 正在为策略 'pbroe6.2_pure_resid_2y' 构建持仓...
     已为 228 个调仓日构建投资组合。
  -> 正在为策略 'pbroe6.2_pure_resid_3y' 构建持仓...
     已为 216 个调仓日构建投资组合。
  -> 正在为策略 'pbroe6.2_pure_resid_5y' 构建持仓...
     已为 192 个调仓日构建投资组合。

构建投资组合完成。所有策略共涉及 228 个唯一的调仓日期。

--- 步骤 2: 执行多策略向量化回测 (并行加速) ---
使用 192 个CPU核心进行并行回测。
回测完成，已为 3 个策略生成 180 条月度收益记录。

--- 步骤 3: 计算并保存各策略绩效指标 ---
月度收益数据已保存至: E:\PBROE\ch6\pbroe6.2_pure_residual_returns.csv
格式化的绩效评估报告已保存至: E:\PBROE\ch6\pbroe6.2_pure_residual_performance.csv

--- 各策略绩效对比简报 ---
                         年化收益率   年化波动率  夏普比率     最大回撤    年化换手率    累计收益率  \
策略名称                                                                      
pbroe6.2_pure_resid_2y  14.24%  25.53%  0.44  -33.80%  197.81%  636.40%   
pbroe6.2_pure_resid_3y  13.67%  25.51%  0.42  -34.00%  193.89%  583.04%   
pbroe6.2_pure_resid_5y  13.18%  25.57%  0.40  -34.29%  186.52%  540.68%   
基准 (沪深300)               1.39%       -     -   